**Downloading and Cleaning Meterological Data** <br>
Carolyn Vilter <br>
<br>
Inspiration / Meteorological API approach from Varsha Gopalakrishnan<br>
https://github.com/varsha2509/hyperlocal-aq-prediction/blob/master/Notebooks/Daymet-Data-API-Call.ipynb

In [2]:
# Setup
import daymetpy
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np

/opt/homebrew/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [7]:
# Basis of eventual met dataset: Vishal's air quality sensor data
# Air quality sensors will be the rows/obs of the final dataset

# Other users: change pathname if needed
air_quality_NO2 = pd.read_csv("/Users/carolynvilter/Documents/MSCAPP_Y1Q3/machine_learning/air-quality-prediction/data/cleaned/air_quality_NO2.csv")
air_quality_NO2

,Unnamed: 0,locationId,location,city,country,utc,local,parameter,value,unit,latitude,longitude
0,0,61509,NaN,Houston,US,2018-03-31T06:00:00+00:00,2018-03-31T01:00:00-05:00,no2,0.000517,ppm,29.789446,-95.483822
1,1,61509,NaN,Houston,US,2018-03-31T06:00:00+00:00,2018-03-31T01:00:00-05:00,no2,0.000756,ppm,29.750588,-95.419853
2,2,61509,NaN,Houston,US,2018-03-31T06:00:00+00:00,2018-03-31T01:00:00-05:00,no2,0.001427,ppm,29.810590,-95.348540
3,3,61509,NaN,Houston,US,2018-03-31T06:00:00+00:00,2018-03-31T01:00:00-05:00,no2,0.001262,ppm,29.786917,-95.298818
4,4,61509,NaN,Houston,US,2018-03-31T06:00:00+00:00,2018-03-31T01:00:00-05:00,no2,0.000803,ppm,29.604903,-95.364682
...,...,...,...,...,...,...,...,...,...,...,...,...
11529,11529,61509,NaN,Houston,US,2018-03-31T06:00:00+00:00,2018-03-31T01:00:00-05:00,no2,0.000631,ppm,29.806077,-95.393692
11530,11530,61509,NaN,Houston,US,2018-03-31T06:00:00+00:00,2018-03-31T01:00:00-05:00,no2,0.000593,ppm,29.735933,-95.235322
11531,11531,61509,NaN,Houston,US,2018-03-31T06:00:00+00:00,2018-03-31T01:00:00-05:00,no2,0.001280,ppm,29.770637,-95.397326
11532,11532,61509,NaN,Houston,US,2018-03-31T06:00:00+00:00,2018-03-31T01:00:00-05:00,no2,0.001794,ppm,29.801806,-95.341444


In [13]:
# FIRST: tf_no2 data
# Create lat and lon rounded to 2 dp - the sensitivity of the met data API
air_quality_NO2["lat_short"] = air_quality_NO2["latitude"].round(2)
air_quality_NO2["lon_short"] = air_quality_NO2["longitude"].round(2)

# Create a unique list of lat, long pairs rounded to 2 decimal places
# These are the points we need meteorological data for 
air_quality_NO2_short = air_quality_NO2[["lat_short", "lon_short"]]
air_quality_NO2_short.drop_duplicates(inplace = True)
air_quality_NO2_short

/var/folders/w0/hp1p0mz546x0rnc9mft_s83w0000gn/T/ipykernel_82138/4108640535.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  air_quality_NO2_short.drop_duplicates(inplace = True)


,lat_short,lon_short
0,29.79,-95.48
1,29.75,-95.42
2,29.81,-95.35
3,29.79,-95.30
4,29.60,-95.36
...,...,...
3673,29.77,-95.26
3987,29.78,-95.46
4610,29.79,-95.49
5027,29.62,-95.36


In [ ]:

# for each lat, lon pair in the list, get 2017-2018 data

# then limit to the time range (sept 2017 to april 2018, inclusive?)

# then compress to one row, all obs - averages

# then add lat lon columns

# then append the new row to the main df

# then merge the new, main met df onto vishal's data


# example
houston_loc = (-95.38, 29.77)
houston2 = daymetpy.daymet_timeseries(lon=houston_loc[0], lat=houston_loc[1], start_year=2017, end_year=2018)